# Registration workshop 1
## 17th January 2019
### Python version

[Open in Colab](https://colab.research.google.com/github/meganzoe/ipmi-registration/blob/master/registration_workshop_1_example_solution.ipynb)

Required packages: `scikit-image` , `scipy`($\geqslant$0.14), `numpy`, `matplotlib` if you'd like to run this notebook or the `utils.py` functions locally. 

First we will set up colab so that the data and `utils.py` script can be loaded. If you are not running this notebook through Colab you can comment out this first block.

In [ ]:
!git clone https://github.com/meganzoe/ipmi-registration
import os
os.chdir('ipmi-registration')

### Loading images
Frist load the 2D lung MRI stored in lung_MRI_slice.png using the python library: `scikit-image`. The image will be stored as a `numpy` array object. `numpy` is a fundamental package for scientific computing in Python.

In [ ]:
import skimage.io
img = skimage.io.imread('lung_MRI_slice.png')

The image is stored as unsigned 8-bit integers:

In [ ]:
print(img.dtype)

You should convert `img` to double precision so that errors do not occur when processing the image due to the limited precision. 

In [ ]:
import numpy as np
# convert img to double

Numpy arrays used 'matrix' coordinates - the first cooridnate is the row number (i.e. the y coordinate) and the second is the column number (i.e. the x coordinate). The rows are also numbered from top to bottom.
We need to reoritentate the image - from (y-axis 1st dim, 0 at top) to
'standard orientation' (y-axis 2nd dim, 0 at bottom) - so that it can be dispalyed 
using the `dispImage` function with the origin in the lower left.

When the image is read in using `scikit-image`'s `imread` function it is initially stored in ‘ij’ coordinates – therefore, before proceeding it is necessary to re-orientate it into ‘standard orientation’. This can be done by first flipping along the second dimension (moving the first pixel from the top to the bottom of the image) and taking the transpose of the matrix (switching x and y dimensions). We can do this easily using 2 handy functions from `numpy`:

In [ ]:
img = np.flipud(img)
img = img.T

Now view the image using the `dispImage` function from `utils.py` which should be in the same folder as this notebook. 

In [ ]:
from utils import dispImage
dispImage(img)

Create an affine matrix representing a translation by 10 pixels in the `x` direction and 20 pixels in the `y` direction. You can declare this using `numpy` matrices:

In [ ]:
T = np.matrix([[1, 0, 0],[0,1,0],[0,0,1]])
print(T)

Create a deformation field for the transformation using the provided `defFieldFromAffineMatrix` function, and then resample the image with the deformation field using the provided `resampImageWithDefField` function within `utils.py`. 

First, calculate the deformation field from the translation matrix.

In [ ]:
from utils import defFieldFromAffineMatrix, resampImageWithDefField

num_pix_x, num_pix_y = img.shape
# calculate def_field here

Now resample and display the image. The function `resampImageWithDefField` uses linear interpolation by default; other supported methods for `scipy`'s `interpn` function are 'nearest' and 'splinef2d'.

Does it appear as expected? Note, the `resampleImageWithDefField` function uses pull-interpolation, so the image will appear to have been transformed by the inverse of the transformation in the affine matrix. 

In [ ]:
# calculate the resampled image and display it here

Check what value has been assigned to pixels that were originally outside the image. This is known as the ‘padding value’ or ‘extrapolation value’. A value of NaN (not a number) is often used to indicate that the true value for these pixels is unknown, and therefore they should be ignored when calculating similarity measures during image registration.

In [ ]:
# print the value of the resampled image in the top right corner
# e.g. print(img_resampled[-1,-1])

Resample the image again using nearest neighbour interpolation and spline interpolation and display the resulting images. Do the different interpolation methods give different results? It may be useful to use difference images (one minus the other) to assess this. 

What about if you now use a translation of **10.5** pixels in the x direction, and **20.5** pixels in the y direction?
Make sure you understand why you get the results that you do (if you are not sure ask one of the lab assistants).

Note – the transformed images may not have exactly the same intensity ranges as the original images due to interpolation and pixels moving outside of the image. This can cause unintentional differences in appearance if the images are displayed using their full intensity ranges (which is the default behaviour with the `dispImage` function). Therefore, it is often a good idea to ensure exactly the same intensity range is used when displaying and comparing different images (e.g. the intensity range of the original image). This can be done using the `int_lims` input to the dispImage function.

Write a function that will calculate the affine matrix corresponding to a rotation about a point, P. The inputs to the function should be the angle of rotation (in degrees) and the coordinates of the point, i.e.:

In [ ]:
def affineMatrixForRotationAboutPoint(theta, p_coords):
  """
  function to calculate the affine matrix corresponding to an anticlockwise
  rotation about a point
  
  INPUTS:    theta: the angle of the rotation, specified in degrees
             p_coords: the 2D coordinates of the point that is the centre of
                 rotation. p_coords[0] is the x coordinate, p_coords[1] is
                 the y coordinate
  
  OUTPUTS:   aff_mat: a 3 x 3 affine matrix
  """
  return R

Use the above function to calculate the affine matrix for an anticlockwise rotation of 5 degrees about the centre of the image.

Note – the image has an even number of pixels in each dimension, so the centre of the image will not be the centre of a pixel. The width and height of the image referred to in the lecture slides are the width and height from the centre of the first pixel to the centre of the last pixel, i.e. width = number of pixels in x – 1, height = number of pixels in y – 1.

Create a deformation field from the affine matrix above and use it to resample the image using linear interpolation. Now apply the same transformation to the transformed image and display the result. Repeat this 72 times so that the image will appear to rotate a full 360 degrees.
Display the results at periodic intervals using the `dispImg` function or create your own `matplotlib.pyplot`.  
What do you notice? 

You will notice that the image gets smaller and smaller as it rotates. This is because of the NaN padding values – when a pixel value is interpolated from one or more NaN values it also gets set to NaN, so the pixels at the edge of the image keep getting set to NaN, and the image gets smaller after each rotation. 

To prevent this replace the NaN values in the transformed image with 0s before applying the next rotation (this is effectively using a padding value of 0 rather than NaN). YOu can do this using the `numpy` `isnan()` function.

You will notice that the corners of the image still get ‘rounded off’ as it rotates so that it has become a circle after rotating 90 degrees. Do you understand why this happens?

Now repeat the above using nearest neighbour and spline interpolation and compare the final results to the result obtained using linear interpolation.
Now experiment with using different angles (both smaller and larger) and rotating about a different point.
Make sure you understand all the results you get.

The blurring artefacts and the ‘rounding off’ of the images seen above are caused by multiple re-samplings of the image. This can be prevented by composing the rotations and applying the resulting transformation to the original image instead of the transformed image. Use this approach to create animations of the rotating image as above, but which do not suffer from blurring artefacts or ‘rounding off’ of the images. Try this using nearest neighbour, linear, and spline interpolation.

As discussed in the lectures, it is possible to resample an image using push-interpolation, but it is far less computationally efficient than using pull-interpolation.

Create an affine matrix representing a rotation by -30 degrees about the point 150,150, and use this to calculate the corresponding deformation field. Use the `resampImageWithDefFieldPushInterp` function to create the transformed image using push-interpolation. Is the result as expected?

Now use pull-interpolation (i.e. the `resampImageWithDefField` function) to create the same result. Compare the results – you should notice that they appear very similar, but if you display a difference image between the results you will see some small differences.

The main difference is in the computation time. Re-create the rotating image sequences above but using push-interpolation and you will notice the difference!